# Análise da Motivação dos Alunos de Engenharia da Computação da Universidade Estadual do Maranhão (UEMA)

Análise exploratória dos dados da pesquisa sobre
a participação em laboratórios/projetos e motivação
dos alunos de Engenharia da Computação.

Dados:
- alunos_geral.csv
- bolsistas.csv

Objetivo:
Explorar, descrever e comparar percepções de alunos
participantes e não participantes de atividades extracurriculares.

Link repositório Github: https://github.com/Gabriel-c0Nsp/analise-motivacao

### Dependências

In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Leitura dos CSVs

In [91]:
alunos = pd.read_csv("dados/alunos_geral.csv")
pesquisadores = pd.read_csv("dados/bolsistas.csv")

### Remoção de espaços em branco

In [92]:
alunos.columns = alunos.columns.str.strip()
bolsistas.columns = bolsistas.columns.str.strip()

### Funções auxiliares

### Setup

In [93]:
LIKERT_5 = {
    "Discordo totalmente": 1,
    "Discordo parcialmente": 2,
    "Nem concordo nem discordo": 3,
    "Neutro": 3,
    "Concordo parcialmente": 4,
    "Concordo totalmente": 5,
}

SIM_NAO = {"Não": 0, "Sim": 1}

COL = {
    # alunos (geral)
    "al_participa_lab": "Participa de algum laboratório ou projeto de pesquisa da universidade?",
    "al_lab_ajuda": "A participação em laboratórios ou projetos de pesquisa ajuda (ou ajudaria) a compreender melhor os conteúdos das disciplinas.",
    "al_fin_afeta": "As dificuldades financeiras afetam minha vida acadêmica.",
    "al_trocar_curso": "Considera trocar de curso?",
    "al_acolh_prof": "Me sinto acolhido(a) por professores e monitores quando tenho dúvidas.",

    # pesquisadores
    "pes_participacao": "Qual a sua participação em relação às atividades citadas acima?",
    "pes_falta_tempo": "Sinto que não tenho tempo suficiente para conciliar as disciplinas regulares, a atividade no laboratório/projeto e minha vida pessoal.",
    "pes_pensou_desistir": "Já pensou em desistir da atividade de laboratório ou extensão por excesso de demandas acadêmicas?",
    "pes_orientacao": "Recebo orientação adequada do meu professor orientador.",
    "pes_recomendaria": "Você já recomendou a colegas a participação no laboratório ou projeto?",
    "pes_reconhecimento_motiv": "O reconhecimento (formal ou informal) pelo meu trabalho no laboratório/projeto contribuiu para minha motivação no curso.",
    "pes_lab_ajuda": "A participação em projetos de pesquisa ou laboratórios ajudou a compreender melhor os conteúdos vistos em sala de aula.",
}

In [94]:
def clean_text(x):
    """Padroniza strings: remove espaços extras e converte para string quando possível."""
    if pd.isna(x):
        return x
    return str(x).strip()

def freq_table(df, col, dropna=False, sort_index=True):
    """
    Retorna uma tabela com contagem e percentual da coluna.
    - dropna=False mostra NaN
    """
    s = df[col].map(clean_text)
    counts = s.value_counts(dropna=dropna)
    if sort_index:
        counts = counts.sort_index()
    perc = (counts / counts.sum() * 100).round(1)
    return pd.DataFrame({"count": counts, "percent": perc})

## Uso ##
# col = "A participação em projetos de pesquisa ou laboratórios ajudou a compreender melhor os conteúdos vistos em sala de aula."
# freq_table(participantes, col)

def to_likert_num(df, col, new_col):
    """
    Cria new_col com valores 1..5 a partir de uma coluna Likert.
    Também retorna a contagem de NaN para você checar se algo não foi mapeado.
    """
    s = df[col].map(clean_text)
    df[new_col] = s.map(LIKERT_5)
    nan_count = df[new_col].isna().sum()
    return nan_count

## Uso ##
# col = "A participação em projetos de pesquisa ou laboratórios ajudou a compreender melhor os conteúdos vistos em sala de aula."
# nan = to_likert_num(participantes, col, "ajuda_compreender_num")
# nan

def to_bin_num(df, col, new_col):
    s = df[col].map(clean_text)
    df[new_col] = s.map(SIM_NAO)
    nan_count = df[new_col].isna().sum()
    return nan_count

## Uso ##
# col = "Participa de algum laboratório ou projeto de pesquisa da universidade?"
# nan = to_bin_num(alunos, col, "participa_lab")
# nan

def plot_likert_bar(df, col_num, title):
    counts = df[col_num].value_counts().sort_index()
    counts.plot(kind="bar")
    plt.xlabel("Likert (1=Discordo total, 5=Concordo total)")
    plt.ylabel("Número de respostas")
    plt.title(title)
    plt.show()

## Uso ##
# plot_likert_bar(participantes, "ajuda_compreender_num", "Participantes: ajuda a compreender conteúdos")

def crosstab_counts(df, a, b):
    """Tabela de contagens: A x B."""
    return pd.crosstab(df[a], df[b])

def crosstab_rowperc(df, a, b):
    """Tabela de percentuais por linha (cada linha soma 100%)."""
    tab = pd.crosstab(df[a], df[b])
    return (tab.div(tab.sum(axis=1), axis=0) * 100).round(1)

## Uso ##
# 1) garantir colunas numéricas
# to_bin_num(alunos, "Participa de algum laboratório ou projeto de pesquisa da universidade?", "participa_lab")
# to_likert_num(alunos, "A participação em laboratórios ou projetos de pesquisa ajuda (ou ajudaria) a compreender melhor os conteúdos das disciplinas.", "lab_ajuda_num")

# 2) cruzar
# crosstab_counts(alunos, "participa_lab", "lab_ajuda_num")
# crosstab_rowperc(alunos, "participa_lab", "lab_ajuda_num")

# Diferentes testes e análises exploratórias a partir daqui (temporário)

### Criação de colunas para análise

In [95]:
# Alunos: participa_lab × lab_ajuda_compreender
to_bin_num(alunos, COL["al_participa_lab"], "participa_lab")
to_likert_num(alunos, COL["al_lab_ajuda"], "lab_ajuda_num")

# Alunos: dificuldades_financeiras_afetam × considera_trocar_curso
to_likert_num(alunos, COL["al_fin_afeta"], "fin_afeta_num")
to_bin_num(alunos, COL["al_trocar_curso"], "trocar_curso")

# Alunos: acolhimento_professores × considera_trocar_curso
to_likert_num(alunos, COL["al_acolh_prof"], "acolh_prof_num")
# trocar_curso já criado acima

# Pesquisadores: falta_tempo_conciliar × pensou_desistir
to_likert_num(pesquisadores, COL["pes_falta_tempo"], "falta_tempo_num")
to_bin_num(pesquisadores, COL["pes_pensou_desistir"], "pensou_desistir")

# Pesquisadores: orientacao_orientador × recomendaria
to_likert_num(pesquisadores, COL["pes_orientacao"], "orientacao_num")
to_bin_num(pesquisadores, COL["pes_recomendaria"], "recomendaria")

# Pesquisadores: reconhecimento_motivacao × recomendaria
to_likert_num(pesquisadores, COL["pes_reconhecimento_motiv"], "reconhecimento_num")
# recomendaria já criado acima

# Pesquisadores: participacao (bolsista/voluntario) × pensou_desistir
pesquisadores["participacao"] = pesquisadores[COL["pes_participacao"]].map(clean_text)
# pensou_desistir já criado acima

# Ponte: comparar “laboratórios ajudam” entre formulários
to_likert_num(pesquisadores, COL["pes_lab_ajuda"], "lab_ajuda_num")

np.int64(0)